## <font color='black'> Análise e Limpeza de Dados de Telecomunicações </font>

Uma empresa de telecomunicações possui registros históricos em um volume dedados de  mais  de  150  mil  registros.  Os  dados  serão  usados  para  modelagem  preditiva,  mas  antes vou realizar um trabalho de limpeza pois os dados possuem diversos problemas.

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.15


In [2]:
# Imports
import math
import sys, os
import numpy as np
import pandas as pd

In [76]:
#caminho para os módulos Python
sys.path.append(os.path.abspath(os.path.join('modulos')))
from estrategia1 import *
from estrategia2 import *
from estrategia3 import *

### Carregando os Dados

In [4]:
#lista para identificar valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

In [5]:
#Carrega o dataset
dataset = pd.read_csv("dados/dataset.csv", na_values = lista_labels_valores_ausentes)

In [6]:
dataset.shape

(150001, 55)

In [7]:
dataset.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [8]:
#dicionário de dados
dicionario = pd.read_excel("dados/Dicionario.xlsx")

In [9]:
dicionario.shape

(56, 2)

In [10]:
dicionario.head()

,Fields,Description
0,bearer id,xDr session identifier
1,Dur. (ms),Total Duration of the xDR (in ms)
2,Start,Start time of the xDR (first frame timestamp)
3,Start ms,Milliseconds offset of start time for the xDR ...
4,End,End time of the xDR (last frame timestamp)


#### Exploração dos Dados

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [12]:
dataset.describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500010e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710569e+06,6.345423e+06,6.725218e+06,6.359490e+06,2.439675e+08,4.782700e+06,2.432050e+08,4.769004e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833501e+06,5.517965e+06,5.777156e+06,5.475981e+06,2.104733e+08,4.128476e+06,2.101869e+08,4.145943e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161602e+07,1.101345e+07,1.164222e+07,1.099638e+07,4.234081e+08,8.291208e+06,4.218030e+08,8.267071e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744852e+07,1.651556e+07,1.747048e+07,1.650727e+07,6.331742e+08,1.243162e+07,6.316918e+08,1.238415e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


In [13]:
dataset.shape

(150001, 55)

In [14]:
dicionario.shape

(56, 2)

Existem 15001 linhas e 55 colunas no dataframe,

No dicionario tem 56 colunas. 

Existe divergencia nesses numeros, uma coluna faltante neste dataset.

***Verificação***

In [15]:
#Compararar as colunas

df_compara = pd.concat([pd.Series(dataset.columns.tolist()),
                      dicionario["Fields"]], 
                      axis = 1)

In [16]:
df_compara.head()

,0,Fields
0,Bearer Id,bearer id
1,Start,Dur. (ms)
2,Start ms,Start
3,End,Start ms
4,End ms,End


In [17]:
df_compara.rename(columns ={0:'Colunas no Dataset', 'Fields': 'Colunas no Dicionario'}, inplace = True)

In [18]:
df_compara

,Colunas no Dataset,Colunas no Dicionario
0,Bearer Id,bearer id
1,Start,Dur. (ms)
2,Start ms,Start
3,End,Start ms
4,End ms,End
5,Dur. (ms),End ms
6,IMSI,Dur. (s)
7,MSISDN/Number,IMSI
8,IMEI,MSISDN/Number
9,Last Location Name,IMEI


"Dur. (Ms)" é ignorado no dataset como visto no índice 1.

O mesmo nome de coluna "Dur. (Ms)" aparece no dataset no índice 5.

Enquanto o arquivo de dicionário nos diz que é "Dur. (S)" no índice 6

Vou verificar qual é o valor certo das variáveis.

In [20]:
dataset[['Dur. (ms)', 'Dur. (ms).1']]

,Dur. (ms),Dur. (ms).1
0,1823652.0,1.823653e+09
1,1365104.0,1.365104e+09
2,1361762.0,1.361763e+09
3,1321509.0,1.321510e+09
4,1089009.0,1.089009e+09
...,...,...
149996,81230.0,8.123076e+07
149997,97970.0,9.797070e+07
149998,98249.0,9.824953e+07
149999,97910.0,9.791063e+07


As duas colunas tem os mesmo valor mas com unidades diferentes. Vou renomear as colunas

In [21]:
# Renomeia colunas
dataset.rename(columns = {'Dur. (ms)': 'Dur (s)', 
                          'Dur. (ms).1': 'Dur (ms)', 
                          'Start ms': 'Start Offset (ms)', 
                          'End ms': 'End Offset (ms)'}, 
               inplace = True)

In [24]:
dataset.columns.tolist()

['Bearer Id',
 'Start',
 'Start Offset (ms)',
 'End',
 'End Offset (ms)',
 'Dur (s)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur (ms)',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 12

#### Tratamento dos Dados

In [25]:
func_calc_percentual_valores_ausentes(dataset)

O dataset tem 12.72 % de valores ausentes.


In [26]:
tabela_de_valores_ausentes = func_calc_percentual_valores_ausentes_coluna(dataset)

O dataset tem 55 colunas.
Encontrado: 41 colunas que têm valores ausentes.


In [27]:
tabela_de_valores_ausentes

,Valores Ausentes,% de Valores Ausentes,Dtype
Nb of sec with 37500B < Vol UL,130254,86.84,float64
Nb of sec with 6250B < Vol UL < 37500B,111843,74.56,float64
Nb of sec with 125000B < Vol DL,97538,65.02,float64
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
Nb of sec with 31250B < Vol DL < 125000B,93586,62.39,float64
Nb of sec with 1250B < Vol UL < 6250B,92894,61.93,float64
Nb of sec with 6250B < Vol DL < 31250B,88317,58.88,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
HTTP UL (Bytes),81810,54.54,float64
HTTP DL (Bytes),81474,54.32,float64


#### Remoção de colunas com valores ausentes

In [31]:
colunas_para_remover = tabela_de_valores_ausentes[tabela_de_valores_ausentes['% de Valores Ausentes'] >= 30.00].index.tolist()

In [32]:
colunas_para_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'TCP UL Retrans. Vol (Bytes)',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'TCP DL Retrans. Vol (Bytes)',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

A variável 'TCP UL Retrans. Vol (Bytes)' e 'TCP DL Retrans. Vol (Bytes)' provavelmente é importante. TCP(protocolo de controle de transmissão)

In [33]:
colunas_para_remover = [col for col in colunas_para_remover if col not in ['TCP UL Retrans. Vol (Bytes)',
    'TCP DL Retrans. Vol (Bytes)']]

In [34]:
colunas_para_remover

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [35]:
dataset_limpo = dataset.drop(colunas_para_remover, axis = 1)

In [36]:
dataset_limpo.shape

(150001, 47)

#### Verificação de valores Ausentes

In [37]:
func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 3.85 % de valores ausentes.


In [38]:
func_calc_percentual_valores_ausentes_coluna(dataset_limpo)

O dataset tem 47 colunas.
Encontrado: 33 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
TCP UL Retrans. Vol (Bytes),96649,64.43,float64
TCP DL Retrans. Vol (Bytes),88146,58.76,float64
Avg RTT DL (ms),27829,18.55,float64
Avg RTT UL (ms),27812,18.54,float64
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64


In [39]:
#Vou aplicar preenchimento reverso nas variáveis TCP.(imputação de valores ausentes)
fix_missing_bfill(dataset_limpo,'TCP UL Retrans. Vol (Bytes)')

96649 valores ausentes na coluna TCP UL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         7230.0
1         7230.0
2         7230.0
3         7230.0
4         7230.0
           ...  
149996       NaN
149997       NaN
149998       NaN
149999       NaN
150000       NaN
Name: TCP UL Retrans. Vol (Bytes), Length: 150001, dtype: float64

In [41]:
#Vou aplicar preenchimento reverso nas variáveis TCP.(imputação de valores ausentes)
fix_missing_bfill(dataset_limpo,'TCP DL Retrans. Vol (Bytes)')

5 valores ausentes na coluna TCP DL Retrans. Vol (Bytes) foram substituídos usando o método de preenchimento reverso.


0         19520.0
1         19520.0
2         19520.0
3         19520.0
4         19520.0
           ...   
149996        NaN
149997        NaN
149998        NaN
149999        NaN
150000        NaN
Name: TCP DL Retrans. Vol (Bytes), Length: 150001, dtype: float64

In [42]:
func_calc_percentual_valores_ausentes_coluna(dataset_limpo)

O dataset tem 47 colunas.
Encontrado: 33 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Avg RTT DL (ms),27829,18.55,float64
Avg RTT UL (ms),27812,18.54,float64
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64
UL TP > 300 Kbps (%),792,0.53,float64
50 Kbps < UL TP < 300 Kbps (%),792,0.53,float64


A Variavél 'Avg RTT DL (ms)' e 'Avg RTT UL (ms)' têm uma porcentagem de 18% de valores ausentes. Antes de fazer a imputação de valores, vou verificar a simetria entre as variáveis.

In [43]:
dataset_limpo['Avg RTT DL (ms)'].skew(skipna= True)

62.90782807995961

In [44]:
dataset_limpo['Avg RTT UL (ms)'].skew(skipna= True)

28.45741458546382

Os dados estão altamente enviesados( resultado > 1). 

Não seguem uma distribuição normal(Não estão simetricas e neste caso não posso usar metódos estatistícos).

Vou aplicar o metodo progressivo nas variáveis.

In [45]:
fix_missing_ffill(dataset_limpo,'Avg RTT DL (ms)')

27829 valores ausentes na coluna Avg RTT DL (ms) foram substituídos usando o método de preenchimento progressivo.


0         42.0
1         65.0
2         65.0
3         65.0
4         65.0
          ... 
149996    32.0
149997    27.0
149998    43.0
149999    37.0
150000    37.0
Name: Avg RTT DL (ms), Length: 150001, dtype: float64

In [46]:
fix_missing_ffill(dataset_limpo,'Avg RTT UL (ms)')

27812 valores ausentes na coluna Avg RTT UL (ms) foram substituídos usando o método de preenchimento progressivo.


0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
         ... 
149996    0.0
149997    2.0
149998    6.0
149999    5.0
150000    5.0
Name: Avg RTT UL (ms), Length: 150001, dtype: float64

In [48]:
func_calc_percentual_valores_ausentes_coluna(dataset_limpo)

O dataset tem 47 colunas.
Encontrado: 31 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64
UL TP > 300 Kbps (%),792,0.53,float64
50 Kbps < UL TP < 300 Kbps (%),792,0.53,float64
10 Kbps < UL TP < 50 Kbps (%),792,0.53,float64
UL TP < 10 Kbps (%),792,0.53,float64


In [49]:
func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 0.44 % de valores ausentes.


In [50]:
func_calc_percentual_valores_ausentes_linha(dataset_limpo)

7.96 % das linhas no conjunto de dados contêm pelo menos um valor ausente.


In [51]:
func_calc_percentual_valores_ausentes_coluna(dataset_limpo)

O dataset tem 47 colunas.
Encontrado: 31 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
Handset Type,9559,6.37,object
Handset Manufacturer,9559,6.37,object
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float64
Bearer Id,991,0.66,float64
Nb of sec with Vol UL < 1250B,793,0.53,float64
UL TP > 300 Kbps (%),792,0.53,float64
50 Kbps < UL TP < 300 Kbps (%),792,0.53,float64
10 Kbps < UL TP < 50 Kbps (%),792,0.53,float64
UL TP < 10 Kbps (%),792,0.53,float64


In [52]:
dataset_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Bearer Id                       149010 non-null  float64
 1   Start                           150000 non-null  object 
 2   Start Offset (ms)               150000 non-null  float64
 3   End                             150000 non-null  object 
 4   End Offset (ms)                 150000 non-null  float64
 5   Dur (s)                         150000 non-null  float64
 6   IMSI                            149431 non-null  float64
 7   MSISDN/Number                   148935 non-null  float64
 8   IMEI                            149429 non-null  float64
 9   Last Location Name              148848 non-null  object 
 10  Avg RTT DL (ms)                 150001 non-null  float64
 11  Avg RTT UL (ms)                 150001 non-null  float64
 12  Avg Bearer TP DL

A variável Handset Type e Handset Manufacturer são categórica! Neste caso não posso usar o mesmo metódo, vou preencher com a palavra "Desconhecido"

In [53]:
fix_missing_value(dataset_limpo,"Handset Type",'unknown')

9559 valores ausentes na coluna Handset Type foram substituídos por unknown.


0          Samsung Galaxy A5 Sm-A520F
1         Samsung Galaxy J5 (Sm-J530)
2            Samsung Galaxy A8 (2018)
3                             unknown
4                    Samsung Sm-G390F
                     ...             
149996    Apple iPhone 8 Plus (A1897)
149997        Apple iPhone Se (A1723)
149998        Apple iPhone Xs (A2097)
149999                 Huawei Fig-Lx1
150000                        unknown
Name: Handset Type, Length: 150001, dtype: object

In [54]:
fix_missing_value(dataset_limpo,"Handset Manufacturer",'unknown')

9559 valores ausentes na coluna Handset Manufacturer foram substituídos por unknown.


0         Samsung
1         Samsung
2         Samsung
3         unknown
4         Samsung
           ...   
149996      Apple
149997      Apple
149998      Apple
149999     Huawei
150000    unknown
Name: Handset Manufacturer, Length: 150001, dtype: object

In [55]:
func_calc_percentual_valores_ausentes(dataset_limpo)

O dataset tem 0.17 % de valores ausentes.


In [56]:
func_calc_percentual_valores_ausentes_linha(dataset_limpo)

2.08 % das linhas no conjunto de dados contêm pelo menos um valor ausente.


O dataset tem 2.08 % de valores Ausentes.

2.08% de Valores ausentes não tem um efeito significativo de (150001) linhas no meu dataframe.

Assim,Vou apagar as linhas de valores ausentes.

In [57]:
drop_rows_with_missing_values(dataset_limpo)

3114 linhas contendo valores ausentes foram descartadas.


In [58]:
dataset_limpo.shape

(146887, 47)

In [59]:
func_calc_percentual_valores_ausentes_coluna(dataset_limpo)

O dataset tem 47 colunas.
Encontrado: 0 colunas que têm valores ausentes.


#### Conversão de Tipos de Dados

In [60]:
dataset_limpo.dtypes

Bearer Id                         float64
Start                              object
Start Offset (ms)                 float64
End                                object
End Offset (ms)                   float64
Dur (s)                           float64
IMSI                              float64
MSISDN/Number                     float64
IMEI                              float64
Last Location Name                 object
Avg RTT DL (ms)                   float64
Avg RTT UL (ms)                   float64
Avg Bearer TP DL (kbps)           float64
Avg Bearer TP UL (kbps)           float64
TCP DL Retrans. Vol (Bytes)       float64
TCP UL Retrans. Vol (Bytes)       float64
DL TP < 50 Kbps (%)               float64
50 Kbps < DL TP < 250 Kbps (%)    float64
250 Kbps < DL TP < 1 Mbps (%)     float64
DL TP > 1 Mbps (%)                float64
UL TP < 10 Kbps (%)               float64
10 Kbps < UL TP < 50 Kbps (%)     float64
50 Kbps < UL TP < 300 Kbps (%)    float64
UL TP > 300 Kbps (%)              

Observando o dataframe, a variável Start e End são datas e estão rotuladas como object.

In [62]:
convert_to_datetime(dataset_limpo,['Start'])

In [63]:
convert_to_datetime(dataset_limpo,['End'])

#### String

In [65]:
strig_columns = dataset_limpo.select_dtypes(include= 'object').columns.tolist()
strig_columns

['Last Location Name', 'Handset Manufacturer', 'Handset Type']

In [66]:
convert_to_string(dataset_limpo,strig_columns)

#### Conversão Inteiros

In [68]:
#Lista de Colunas
inteiros_coluna = ['Bearer Id', 'IMSI', 'MSISDN/Number', 'IMEI']

In [69]:
convert_to_int(dataset_limpo,inteiros_coluna)

#### Registros duplicados

In [70]:
drop_duplicates(dataset_limpo)

Nenhuma linha duplicada foi encontrada.


Temos duas colunas que parecem ter a mesma informação com escala diferente.

In [71]:
# Conversão e comparação
temp_df = dataset_limpo[['Dur (s)', 'Dur (ms)']].copy()

multiply_by_factor(temp_df, ['Dur (ms)'], 1/1000)

temp_df['comparison'] = (temp_df['Dur (s)'] == temp_df['Dur (ms)'].apply(math.floor))

In [72]:
temp_df

,Dur (s),Dur (ms),comparison
0,1823652.0,1823652.892,True
1,1365104.0,1365104.371,True
2,1361762.0,1361762.651,True
3,1321509.0,1321509.685,True
4,1089009.0,1089009.389,True
...,...,...,...
149991,61661.0,61661.729,True
149992,84940.0,84940.610,True
149993,125622.0,125622.433,True
149994,113545.0,113545.185,True


Após a confirmação vou apagar a coluna Dur(s). 

Dur(ms) tem uma unidade válida.

In [75]:
drop_columns(dataset_limpo,["Dur (s)"])

1 coluna foi descartada.


#### Outliers

O boxplot nos ajuda a observar valores extremos.

In [78]:
#Class Outliers
trata_outlier = TrataOutlier(dataset_limpo)

In [80]:
list_col = dataset_limpo.select_dtypes('float64').columns.tolist()

In [81]:
list_col

['Start Offset (ms)',
 'End Offset (ms)',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur (ms)',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Social Media DL (Bytes)',
 'Social Media UL (Bytes)',
 'Google DL (Bytes)',
 'Google UL (Bytes)',
 'Email DL (Bytes)',
 'Email UL (Bytes)',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Netflix DL (Bytes)',
 'Netflix UL (Bytes)',
 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)',
 'Total UL (Bytes)',
 'Total DL (Bytes)']

In [82]:
# Visão geral dos outliers
trata_outlier.getOverview(list_col)

Nome de Coluna,DL TP > 1 Mbps (%),10 Kbps < UL TP < 50 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),Activity Duration DL (ms),Activity Duration UL (ms),Nb of sec with Vol UL < 1250B,Nb of sec with Vol DL < 6250B,UL TP < 10 Kbps (%),TCP DL Retrans. Vol (Bytes),Avg Bearer TP UL (kbps),...,Start Offset (ms),Youtube DL (Bytes),Email UL (Bytes),Email DL (Bytes),Google UL (Bytes),Google DL (Bytes),Social Media UL (Bytes),Social Media DL (Bytes),End Offset (ms),Total DL (Bytes)
Min,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,...,0.0,53.0,2.0,14.0,3.0,207.0,0.0,12.0,0.0,7114041.0
Q1,0.0,0.0,0.0,15418.0,22073.0,107.0,87.0,99.0,24895.0,47.0,...,250.0,5833462.0,233439.0,892573.0,1024371.5,2882586.5,16431.0,898089.0,251.0,243073402.5
Median,0.0,0.0,0.0,39726.0,47180.0,217.0,201.0,100.0,391635.0,63.0,...,499.0,11616334.0,466357.0,1793613.0,2054793.0,5766724.0,32908.0,1793409.0,500.0,455963875.0
Q3,0.0,0.0,1.0,697298.5,611803.0,2466.5,2612.5,100.0,3078146.0,1136.0,...,749.0,17440635.5,700297.5,2689818.0,3088071.5,8625551.0,49326.5,2694640.0,750.0,665783068.0
Max,94.0,98.0,76.0,136536461.0,144911293.0,604122.0,604061.0,100.0,4294425570.0,58613.0,...,999.0,23259098.0,936418.0,3586146.0,4121357.0,11462832.0,65870.0,3586064.0,999.0,902969616.0
IQR,0.0,0.0,1.0,681880.5,589730.0,2359.5,2525.5,1.0,3053251.0,1089.0,...,499.0,11607173.5,466858.5,1797245.0,2063700.0,5742964.5,32895.5,1796551.0,499.0,422709665.5
Lower fence,0.0,0.0,-1.5,-1007402.75,-862522.0,-3432.25,-3701.25,97.5,-4554981.5,-1586.5,...,-498.5,-11577298.25,-466848.75,-1803294.5,-2071178.5,-5731860.25,-32912.25,-1796737.5,-497.5,-390991095.75
Upper fence,0.0,0.0,2.5,1720119.25,1496398.0,6005.75,6400.75,101.5,7658022.5,2769.5,...,1497.5,34851395.75,1400585.25,5385685.5,6183621.5,17239997.75,98669.75,5389466.5,1498.5,1299847566.25
Skew,5.345557,10.941071,4.503234,5.821286,7.321154,7.467915,9.009273,-8.958674,16.755094,4.473627,...,0.000823,0.000119,0.007312,-0.002659,0.002242,-0.008414,0.000258,-0.001633,-0.001251,-0.003579
Num_Outliers,36594,31509,29343,26126,25501,24870,24604,21811,21625,21104,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# Replace dos outliers
trata_outlier.replace_outliers_with_fences(list_col)

In [84]:
trata_outlier.getOverview(list_col)

Nome de Coluna,Start Offset (ms),Nb of sec with Vol DL < 6250B,Social Media DL (Bytes),Social Media UL (Bytes),Google DL (Bytes),Google UL (Bytes),Email DL (Bytes),Email UL (Bytes),Youtube DL (Bytes),Youtube UL (Bytes),...,DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),DL TP > 1 Mbps (%),UL TP < 10 Kbps (%),10 Kbps < UL TP < 50 Kbps (%),50 Kbps < UL TP < 300 Kbps (%),UL TP > 300 Kbps (%),Activity Duration DL (ms),Total DL (Bytes)
Min,0.0,1.0,12.0,0.0,207.0,3.0,14.0,2.0,53.0,105.0,...,77.5,0.0,0.0,0.0,97.5,0.0,0.0,0.0,0.0,7114041.0
Q1,250.0,87.0,898089.0,16431.0,2882586.5,1024371.5,892573.0,233439.0,5833462.0,5516549.0,...,91.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,15418.0,243073402.5
Median,499.0,201.0,1793409.0,32908.0,5766724.0,2054793.0,1793613.0,466357.0,11616334.0,11013447.0,...,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,39726.0,455963875.0
Q3,749.0,2612.5,2694640.0,49326.5,8625551.0,3088071.5,2689818.0,700297.5,17440635.5,16514278.0,...,100.0,4.0,1.0,0.0,100.0,0.0,0.0,0.0,697298.5,665783068.0
Max,999.0,6400.75,3586064.0,65870.0,11462832.0,4121357.0,3586146.0,936418.0,23259098.0,22011962.0,...,100.0,10.0,2.5,0.0,100.0,0.0,0.0,0.0,1720119.25,902969616.0
IQR,499.0,2525.5,1796551.0,32895.5,5742964.5,2063700.0,1797245.0,466858.5,11607173.5,10997729.0,...,9.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,681880.5,422709665.5
Lower fence,-498.5,-3701.25,-1796737.5,-32912.25,-5731860.25,-2071178.5,-1803294.5,-466848.75,-11577298.25,-10980044.5,...,77.5,-6.0,-1.5,0.0,97.5,0.0,0.0,0.0,-1007402.75,-390991095.75
Upper fence,1497.5,6400.75,5389466.5,98669.75,17239997.75,6183621.5,5385685.5,1400585.25,34851395.75,33010871.5,...,113.5,10.0,2.5,0.0,101.5,0.0,0.0,0.0,1720119.25,1299847566.25
Skew,0.000823,1.205814,-0.001633,0.000258,-0.008414,0.002242,-0.002659,0.007312,0.000119,-0.001265,...,-1.22817,1.315047,1.098748,0.0,-0.947267,0.0,0.0,0.0,1.205281,-0.003579
Num_Outliers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
dataset_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146887 entries, 0 to 149995
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       146887 non-null  int64         
 1   Start                           146887 non-null  datetime64[ns]
 2   Start Offset (ms)               146887 non-null  float64       
 3   End                             146887 non-null  datetime64[ns]
 4   End Offset (ms)                 146887 non-null  float64       
 5   IMSI                            146887 non-null  int64         
 6   MSISDN/Number                   146887 non-null  int64         
 7   IMEI                            146887 non-null  int64         
 8   Last Location Name              146887 non-null  string        
 9   Avg RTT DL (ms)                 146887 non-null  float64       
 10  Avg RTT UL (ms)                 146887 non-null  float64

In [86]:
dataset_limpo.shape

(146887, 46)

Depois de uma ánalise exploratória, tratamento de valores ausente , conversão de tipo de dados ,chegamos ao fim de mais uma ánalise e o dataframe esta pronto para aplicar um modelo.

### FIM